In [41]:
from dotenv import load_dotenv
from os import getenv
import requests
import praw
import pandas as pd

In [42]:
load_dotenv()

APP_NAME = getenv('APP_NAME')
APP_ID = getenv('APP_ID')
APP_SECRET = getenv('APP_SECRET')
USERNAME = getenv('REDDIT_USERNAME')
PASSWORD = getenv('REDDIT_PASSWORD')


In [43]:
print(USERNAME)

Beatrice359


In [44]:
auth = requests.auth.HTTPBasicAuth(APP_ID, APP_SECRET)

data = {
    'grant_type': 'password',
    'username': USERNAME,
    'password': PASSWORD
}

headers = {'User-Agent': f'windows:{APP_NAME}:v1.0 (by /u/{USERNAME})'}

res = requests.post(
    'https://www.reddit.com/api/v1/access_token', # OAuth2
    auth=auth,
    data=data,
    headers=headers
)

print(res.status_code)

200


In [45]:
TOKEN = res.json()['access_token']
headers = {**headers, **{'Authorization': f'bearer {TOKEN}'}}

In [46]:
res = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)
print(res.status_code)

200


In [47]:
res

<Response [200]>

In [48]:
res.json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': True,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '#FF99AA',
  'user_is_muted': None,
  'display_name': 'u_Beatrice359',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://www.redditstatic.com/avatars/defaults/v2/avatar_default_0.png',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size': None,
  'restrict_commenting': False,
  'subscribers': 0,
  'submit_text_label': '',
  'is_default_icon': True,
  'link_flair_pos

In [49]:
res.json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': True,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '#FF99AA',
  'user_is_muted': None,
  'display_name': 'u_Beatrice359',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://www.redditstatic.com/avatars/defaults/v2/avatar_default_0.png',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size': None,
  'restrict_commenting': False,
  'subscribers': 0,
  'submit_text_label': '',
  'is_default_icon': True,
  'link_flair_pos

In [52]:
import requests
import pandas as pd
from datetime import datetime

# function to convert responses to dataframes
def df_from_reddit(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()



    for post in res.json()['data']['children']:
        new_row = {
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


    return df




In [53]:
housing_Price_text = pd.DataFrame()
params = {'limit': 100}


link1 = requests.get("https://oauth.reddit.com/r/ireland/search/?q=housing%20price&restrict_sr=1",
                       headers=headers,
                       params=params)

link2 = requests.get("https://oauth.reddit.com/r/unitedkingdom/search/?q=housing%20price&restrict_sr=1",
                       headers=headers,
                       params=params)

from itertools import cycle

links_list = []
links_list.append(link1)
links_list.append(link2)

num_iterations = 40  # Numero di iterazioni desiderate

# Crea un ciclo infinito sui due link
link_cycle = cycle(links_list)

for _ in range(num_iterations):
    # Prendi il prossimo link dal ciclo
    link = next(link_cycle)

    # Fai la richiesta
    res = link

    # Resto del tuo codice...
    new_df = df_from_reddit(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    # concatenate new_df to construction_Price
    housing_Price_text = pd.concat([housing_Price_text, new_df], ignore_index=True)



In [54]:
housing_Price_text

,subreddit,title,selftext,created_utc,id,kind
0,ireland,Dan O'Brien on Twitter re housing price growth...,,2022-05-16T11:54:09Z,uqt5hf,t3
1,ireland,Is a housing price crash coming?,"I have no idea how most of this stuff works, b...",2022-04-20T13:17:23Z,u7v60i,t3
2,ireland,Irish house prices rise 8.9% in first three mo...,,2023-05-11T11:53:51Z,13ejsvy,t3
3,ireland,Third consecutive month of house price decline...,,2023-05-17T12:16:03Z,13jyrhv,t3
4,ireland,House Prices and Inflation,,2023-03-29T15:43:10Z,125q8tn,t3
...,...,...,...,...,...,...
3995,unitedkingdom,Would you pay £63 for a chicken? The artist wh...,,2021-07-14T19:03:34Z,ok9xwo,t3
3996,unitedkingdom,UK house prices increase at fastest rate since...,,2021-05-01T10:26:03Z,n2exto,t3
3997,unitedkingdom,Yougov: Londoners blame rich foreigners for ho...,,2014-04-13T21:30:29Z,22y4jw,t3
3998,unitedkingdom,Second homes: Man fears house prices could rui...,,2021-11-17T23:35:07Z,qwc1at,t3


In [78]:
housing_Price_text.to_csv('housing_Price_text.csv', index=False)